# 태풍 솔릭(Soulik) 모의를 위한<br/>GFS/GDAS로 부터 ROMS 영역의 S.B.C. 자료 가져오기

GFS 자료는 여러기관에서 다양한 방법으로 제공하고 있으며,<br/>
여기에서는 NCDC의 NOMADS로부터 ftp를 이용하여 grib2 자료를 직접 받는 방법 (2016.09.01 ~ about 3days ago)을 설명합니다.

In [1]:
# Author: Dong-Hoon Kim (http://www.dhkim.info)

%matplotlib inline
import matplotlib.pyplot as plt

import os
from datetime import datetime, timedelta
from urllib import parse, request
from multiprocessing import Process

import numpy as np
from netCDF4 import Dataset, num2date, date2num

## 1. 정보 설정

In [2]:
# 실험명
expname = 'Soulik'; version = 'v4'

# 초기장 날짜 설정
date_beg = datetime(2018,8,18,0) # year, month, day, hour
date_end = datetime(2018,8,25,0) # year, month, day, hour
#date_beg = datetime(2019,6,1,12) # year, month, day, hour
#date_end = datetime(2019,6,1,12) # year, month, day, hour
ymdh_beg = date_beg.strftime('%Y-%m-%dT%H:%M:%SZ')
ymdh_end = date_end.strftime('%Y-%m-%dT%H:%M:%SZ')

#server = 'ftp://ftp.ncep.noaa.gov/pub/data/nccf/com/gfs/prod'
#header = 'gfs'; dataformat = 'pgrb2'; version = '0p50'

delta_hour= 3; time_lead_max = 240 # forecast up to 10 days
tau_list  = range(0,time_lead_max+1,delta_hour)

# ROMS 격자 정보 가져오기
roms_grid_fname = '../Grid/'+expname+'_grd_'+version+'.nc'
extraHoriz  = 2.0 # in degrees. use 2 times of delta X or delta Y.
if not os.path.isfile(roms_grid_fname):
    print("==>Error: not found - %s"%roms_grid_fname)
else:
    with Dataset(roms_grid_fname) as roms_grid:
        minmaxLonR = [round(roms_grid['lon_vert'][:].min()-extraHoriz,1), # west
                      round(roms_grid['lon_vert'][:].max()+extraHoriz,1)] # east
        minmaxLatR = [round(roms_grid['lat_vert'][:].min()-extraHoriz,1), # south
                      round(roms_grid['lat_vert'][:].max()+extraHoriz,1)] # north
        print('==>Info: ', minmaxLonR, minmaxLatR)

==>Info:  [96.3, 192.5] [3.2, 63.6]


## 2. NCDC로부터 GFS 자료받기

* NCDC의 NOMADS로부터 ftp를 이용하여 grib2 자료를 직접 받는 방법 (2016.09.01 ~ about 3days ago)

> **Note**) 2016년 9월부터 최근 자료까지 받으려면 NCDC에서 제공하는 자료를 이용한다.<br/>
> (2016.09.01 ~ 3days ago)

In [ ]:
date_beg = datetime(2018,8,18,0)
date_end = datetime(2018,8,25,0)

dt = date_beg
while(dt <= date_end):
    for tlead in tau_list:
        if tlead > time_lead_max:
            print(C.wrnmsg+'exceeded max time lead (%i/%i)'%(tlead,time_lead_max))
            continue
        if tlead%delta_hour != 0:
            print(C.wrnmsg+'time lead must be every %i hourly but %i'\
                 %(delta_hour,tlead//delta_hour))
            continue
            
        # GFS Forecast (2016.09.01 ~ 3days ago)
        # ex) ftp://nomads.ncdc.noaa.gov/GFS/Grid4/201808/20180818/gfs_4_20180818_1200_240.grb2
        gfsdir = dt.strftime('%Y%m/%Y%m%d')
        gfsfile = 'gfs_4_%s_%3.3i.grb2'%(dt.strftime('%Y%m%d_%H%M'),tlead)
        GFSncdc ='ftp://nomads.ncdc.noaa.gov/GFS/Grid4/%s/%s'%(gfsdir,gfsfile)
        GFSlocal = 'GFS/%s/%s'%(gfsdir,gfsfile)
        
        if os.path.isfile(GFSlocal):
            print('==>Warn: already exists %s'%GFSlocal); continue
        else:
            print('==>Info: %s to %s'%(GFSncdc,GFSlocal))
            locdir = os.path.split(GFSlocal)[0]
            if not os.path.isdir(locdir): os.makedirs(locdir)
                
            try: _=request.urlretrieve(GFSncdc,GFSlocal)
            except Exception as ex: print('==>Error: ',ex)
        
    dt += timedelta(hours=6)

==>Info: ftp://nomads.ncdc.noaa.gov/GFS/Grid4/201808/20180818/gfs_4_20180818_0000_000.grb2 to GFS/201808/20180818/gfs_4_20180818_0000_000.grb2
==>Info: ftp://nomads.ncdc.noaa.gov/GFS/Grid4/201808/20180818/gfs_4_20180818_0000_003.grb2 to GFS/201808/20180818/gfs_4_20180818_0000_003.grb2
==>Info: ftp://nomads.ncdc.noaa.gov/GFS/Grid4/201808/20180818/gfs_4_20180818_0000_006.grb2 to GFS/201808/20180818/gfs_4_20180818_0000_006.grb2
==>Info: ftp://nomads.ncdc.noaa.gov/GFS/Grid4/201808/20180818/gfs_4_20180818_0000_009.grb2 to GFS/201808/20180818/gfs_4_20180818_0000_009.grb2
==>Info: ftp://nomads.ncdc.noaa.gov/GFS/Grid4/201808/20180818/gfs_4_20180818_0000_012.grb2 to GFS/201808/20180818/gfs_4_20180818_0000_012.grb2
==>Info: ftp://nomads.ncdc.noaa.gov/GFS/Grid4/201808/20180818/gfs_4_20180818_0000_015.grb2 to GFS/201808/20180818/gfs_4_20180818_0000_015.grb2
==>Info: ftp://nomads.ncdc.noaa.gov/GFS/Grid4/201808/20180818/gfs_4_20180818_0000_018.grb2 to GFS/201808/20180818/gfs_4_20180818_0000_018.grb2